In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
URL = 'https://www.indeed.com/jobs?q=data+science&l=New+York,+NY&jt=fulltime'
#conducting a request of the stated URL above:
page = requests.get(URL)
#specifying a desired format of “page” using the html parser - this allows python to read the various components of the page, rather than treating it as one long string.
soup = BeautifulSoup(page.text, 'html.parser')
#printing soup in a more structured tree format that makes for easier reading
print(soup.prettify())

<!DOCTYPE html>
<html dir="ltr" lang="en">
 <head>
  <meta content="text/html;charset=utf-8" http-equiv="content-type"/>
  <script src="//d3fw5vlhllyvee.cloudfront.net/s/bc08208/en_US.js" type="text/javascript">
  </script>
  <link href="//d3fw5vlhllyvee.cloudfront.net/s/1ad836e/jobsearch_all.css" rel="stylesheet" type="text/css"/>
  <link href="https://rss.indeed.com/rss?q=data+science&amp;l=New+York%2C+NY&amp;jt=fulltime" rel="alternate" title="Data Science Jobs, Employment in New York, NY" type="application/rss+xml"/>
  <link href="/m/jobs?q=data+science&amp;l=New+York%2C+NY&amp;jt=fulltime" media="only screen and (max-width: 640px)" rel="alternate"/>
  <script type="text/javascript">
   if (typeof window['closureReadyCallbacks'] == 'undefined') {
window['closureReadyCallbacks'] = [];
}

function call_when_jsall_loaded(cb) {
if (window['closureReady']) {
cb();
} else {
window['closureReadyCallbacks'].push(cb);
}
}
  </script>
  <meta content="1" name="ppstriptst"/>
  <script>
   var

In [3]:
def extract_job_title_from_result(soup): 
    jobs = []
    for div in soup.find_all(name= 'div', attrs={'class':'row'}):
        for a in div.find_all(name='a', attrs={'data-tn-element':'jobTitle'}):
            jobs.append(a['title'])
    return(jobs)
extract_job_title_from_result(soup)

['Data Science Manager',
 'DDM – Data Scientist',
 'Principal Data Engineer',
 'Senior Data Engineer',
 'Erwin Data Modeler',
 'Data Science Associate',
 'Data Scientist - Computer Vision',
 'ESG Associate - Data Science & Energy',
 'Associate, Data Science',
 'Data Scientist I',
 'Data Scientist Summer Intern: 2021 (CIO)',
 'Associate Data Analyst',
 'Associate Data Scientist',
 'ESG Data Science and Systems Associate',
 'Data Scientist (P2)',
 'Data Scientist I']

In [4]:
def extract_company_from_result(soup): 
    companies = []
    for div in soup.find_all(name='div', attrs={'class':'row'}):
        company = div.find_all(name='span', attrs={'class':'company'})
        if len(company) > 0:
            for b in company:
                companies.append(b.text.strip())
        else:
            sec_try = div.find_all(name='span', attrs={'class':'result-link-source'})
            for span in sec_try:
                companies.append(span.text.strip())
    return(companies)
 
extract_company_from_result(soup)

['Biz2Credit Inc.',
 'Biz2Credit Inc',
 'Frame AI',
 'Frame AI',
 'Cognizant Technology Solutions',
 'Warner Music Group',
 'Memorial Sloan-Kettering Cancer Center',
 'Link Industrial Properties',
 'Jefferies & Company, Inc.',
 'Guardian Life Insurance Company',
 'IBM',
 'AT&T',
 'AT&T',
 'AllianceBernstein',
 'Memorial Sloan-Kettering Cancer Center',
 'Guardian Life Insurance Company']

In [5]:
def extract_location_from_result(soup): 
    locations = []
    spans = soup.findAll('span', attrs={'class': 'location'})
    for span in spans:
        locations.append(span.text)
    return(locations)
extract_location_from_result(soup)

['Long Island City, NY 11101',
 'New York, NY',
 'New York, NY 10017 (Turtle Bay area)',
 'New York, NY 10016 (Murray Hill area)',
 'New York, NY 10022 (Midtown area)',
 'New York, NY',
 'New York, NY 10001 (Chelsea area)',
 'New York, NY',
 'New York, NY',
 'New York, NY 10020 (Midtown area)']

In [6]:
print(soup)

<!DOCTYPE html>

<html dir="ltr" lang="en">
<head>
<meta content="text/html;charset=utf-8" http-equiv="content-type"/>
<script src="//d3fw5vlhllyvee.cloudfront.net/s/bc08208/en_US.js" type="text/javascript"></script>
<link href="//d3fw5vlhllyvee.cloudfront.net/s/1ad836e/jobsearch_all.css" rel="stylesheet" type="text/css"/>
<link href="https://rss.indeed.com/rss?q=data+science&amp;l=New+York%2C+NY&amp;jt=fulltime" rel="alternate" title="Data Science Jobs, Employment in New York, NY" type="application/rss+xml"/>
<link href="/m/jobs?q=data+science&amp;l=New+York%2C+NY&amp;jt=fulltime" media="only screen and (max-width: 640px)" rel="alternate"/>
<script type="text/javascript">

if (typeof window['closureReadyCallbacks'] == 'undefined') {
window['closureReadyCallbacks'] = [];
}

function call_when_jsall_loaded(cb) {
if (window['closureReady']) {
cb();
} else {
window['closureReadyCallbacks'].push(cb);
}
}
</script>
<meta content="1" name="ppstriptst"/>
<script>
var _scriptDownloadCount = 0;

In [7]:
def extract_summary_from_result(soup): 
    summaries = []
    spans = soup.findAll('div', attrs={'class': 'summary'})
    for span in spans:
        summaries.append(span.text.strip())
    return(summaries)
extract_summary_from_result(soup)

['Manage the data science projects in Agile mode; Prioritize projects based on business impact.\nOverall 10+ years of experience with data science; credit risk…',
 'Work on data projects and proposals involving Biz2Credit’s financial services partners worldwide (banks, non-banks, debt investors, equity investors and others)…',
 'You will collaborate with Frame AI’s data science staff using innovative shared code and test methods that enable continuous delivery of cutting edge research…',
 'You will collaborate with Frame AI’s data science staff using innovative shared code and test methods that enable continuous delivery of cutting edge research…',
 'With the power to apply artificial intelligence and data science to business decisions via enterprise data management solutions, we help leading companies…',
 'The Data Science Associate role is an entry-level position that offers associates with a high-impact introduction to applied data science in the music industry.',
 'Solid understand

In [33]:
max_results_per_city = 100
city_set = ['New+York','Chicago','San+Francisco', 'Austin', 'Seattle', 'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'San+Jose', 'Boston', 'Palo+Alto', 'Denver', 'Houston', 'Miami', 'Washington+DC', 'Arlington']

columns = ['city', 'job_title', 'company_name', 'location', 'summary']

sample_df = pd.DataFrame(columns = columns)

In [9]:
print(city_set)

['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Seattle', 'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh', 'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami', 'Washington+DC', 'Boulder']


In [14]:
def parse_job(page):
    job_post=[]
    #print(page)
    soup = BeautifulSoup(page, 'lxml', from_encoding='utf-8')
    for div in soup.find_all(name='div', attrs={'class':'row'}): 
        
        dictionary={}
        
        job_name = [a.text for a in div.find_all(name='a', attrs={'data-tn-element':'jobTitle'})]
            #job_post.append(a['title']) 
        dictionary["job_name"]='|'.join(job_name)
        
        company = [c.text for c in div.find_all(name='span', attrs={'class':'company'})]
        dictionary['company']='|'.join(company)

        d = div.find_all('div', attrs={'class': 'summary'}) 
        summary = []
        for span in d:
            summary.append(span.text.strip()) 
        dictionary['summary']='|'.join(summary)
        job_post.append(dictionary)
    print(job_post)
    return job_post

In [36]:
project=[]

companies = []
summaries = []
job_titles =[]


for city in city_set[0:30]:
    info={}
    for start in range(0, max_results_per_city, 10):
        url = 'https://www.indeed.com/jobs?q=data+science' + '&l=' + str(city) + '&jt=fulltime' + '&start=' + str(start)
        print(url)
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'html.parser')
        companies.append(extract_company_from_result(soup))
        summaries.append(extract_summary_from_result(soup))
        job_titles.append(extract_job_title_from_result(soup))
        time.sleep(10) 
    
print(f"length of summaries: { len(summaries[0])}")
print(f"length of job_titles: { len(job_titles[0])}")
print(f"length of companies: { len(companies[0])}")

#print(summaries)
#print('___________________________________________________________________________')
#print(job_titles)
#print('___________________________________________________________________________')
#print(companies)
#print('___________________________________________________________________________')

for i in range(len(job_titles)):
    for j in range(len(job_titles[i])):
        try:
            project.append([job_titles[i][j], companies[i][j], summaries[i][j]])
        except:
            #print(e)
            print('companies')
            print(companies[i])
            print('job_titles')
            print(job_titles[i])
            print('summaries')
            print(summaries[i])
        
#print('___________________________________________________________________________')
#print(project)   

project_df = pd.DataFrame.from_records(project)
#print('___________________________________________________________________________')
#print(project_df)

project_df.to_csv('Dataset_NLP.csv', encoding='utf-8')

#saving sample_df as a local csv file — define your own local path to save contents 


https://www.indeed.com/jobs?q=data+science&l=New+York&jt=fulltime&start=0
https://www.indeed.com/jobs?q=data+science&l=New+York&jt=fulltime&start=10
https://www.indeed.com/jobs?q=data+science&l=New+York&jt=fulltime&start=20
https://www.indeed.com/jobs?q=data+science&l=New+York&jt=fulltime&start=30
https://www.indeed.com/jobs?q=data+science&l=New+York&jt=fulltime&start=40
https://www.indeed.com/jobs?q=data+science&l=New+York&jt=fulltime&start=50
https://www.indeed.com/jobs?q=data+science&l=New+York&jt=fulltime&start=60
https://www.indeed.com/jobs?q=data+science&l=New+York&jt=fulltime&start=70
https://www.indeed.com/jobs?q=data+science&l=New+York&jt=fulltime&start=80
https://www.indeed.com/jobs?q=data+science&l=New+York&jt=fulltime&start=90
https://www.indeed.com/jobs?q=data+science&l=Chicago&jt=fulltime&start=0
https://www.indeed.com/jobs?q=data+science&l=Chicago&jt=fulltime&start=10
https://www.indeed.com/jobs?q=data+science&l=Chicago&jt=fulltime&start=20
https://www.indeed.com/jobs?q=

https://www.indeed.com/jobs?q=data+science&l=Boston&jt=fulltime&start=90
https://www.indeed.com/jobs?q=data+science&l=Palo+Alto&jt=fulltime&start=0
https://www.indeed.com/jobs?q=data+science&l=Palo+Alto&jt=fulltime&start=10
https://www.indeed.com/jobs?q=data+science&l=Palo+Alto&jt=fulltime&start=20
https://www.indeed.com/jobs?q=data+science&l=Palo+Alto&jt=fulltime&start=30
https://www.indeed.com/jobs?q=data+science&l=Palo+Alto&jt=fulltime&start=40
https://www.indeed.com/jobs?q=data+science&l=Palo+Alto&jt=fulltime&start=50
https://www.indeed.com/jobs?q=data+science&l=Palo+Alto&jt=fulltime&start=60
https://www.indeed.com/jobs?q=data+science&l=Palo+Alto&jt=fulltime&start=70
https://www.indeed.com/jobs?q=data+science&l=Palo+Alto&jt=fulltime&start=80
https://www.indeed.com/jobs?q=data+science&l=Palo+Alto&jt=fulltime&start=90
https://www.indeed.com/jobs?q=data+science&l=Denver&jt=fulltime&start=0
https://www.indeed.com/jobs?q=data+science&l=Denver&jt=fulltime&start=10
https://www.indeed.com/